In [7]:
import scipy.io as sio   
import argparse
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import r2_score
from scipy.stats.stats import pearsonr
import pickle
from sklearn.linear_model import LinearRegression
import xgboost
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.svm import SVR
import h5py
import pickle

In [8]:
x_train= {}
x_test = {}
y_train = {}
y_test= {}
location_train= {}
location_test= {}
feature_all_train = {}
feature_all_test= {}
aoa_train={}
aoa_test={}
toa_train={}
toa_test={}
rss_train={}
rss_test={}
f = h5py.File('data_cross_250')
for i in range(4):
    x_train[i] = f[str(i)+'/loc_x_train'][:]
    x_test[i] = f[str(i)+'/loc_x_test'][:]
    y_train[i] = f[str(i)+'/loc_y_train'][:] 
    y_test[i] = f[str(i)+'/loc_y_test'][:]
    location_train[i] = f[str(i)+'/loc_train'][:,:]
    location_test[i] = f[str(i)+'/loc_test'][:,:]
    feature_all_train[i] =f[str(i)+'/feature_train'][:,:]
    feature_all_test[i] = f[str(i)+'/feature_test'][:,:]
    aoa_train[i] =f[str(i)+'/aoa_train'][:,:]
    aoa_test[i] = f[str(i)+'/aoa_test'][:,:]
    toa_train[i] =f[str(i)+'/toa_train'][:,:]
    toa_test[i] = f[str(i)+'/toa_test'][:,:]
    rss_train[i] = f[str(i)+'/toa_train'][:,:]
    rss_test[i] = f[str(i)+'/toa_test'][:,:]
aoa_toa_train={}
aoa_toa_test ={}
aoa_toa_rss_train={}
aoa_toa_rss_test ={}
for i in range(4):
    aoa_toa_train[i] = feature_all_train[i][:,7:18]
    aoa_toa_test[i] = feature_all_test[i][:,7:18]
    aoa_toa_rss_train[i] = feature_all_train[i][:,:18]
    aoa_toa_rss_test[i] = feature_all_test[i][:,:18]

In [ ]:
reg = KernelRidge(alpha = 2**(-k)/200.0, kernel = 'rbf',gamma = 2**(-m))

In [18]:
#2**(-8)/200.0
2**(-4)

0.0625

In [ ]:
reg = KernelRidge(alpha = 1.953125*10**(-05)*(1+k/100.0), kernel = 'rbf',gamma = 0.0625*(1+m/100.0))

In [9]:
average_dis_error = {}
for k in range(-10,10):
    average_dis_error[k] = {}
    for m in range(-10,10):
        reg = KernelRidge(alpha = 1.953125*10**(-05)*(1+k/100.0), kernel = 'rbf',gamma = 0.0625*(1+m/100.0))
        regpre_x ={}
        regpre_y = {}
        r2_x = {}
        r2_y = {}
        pcc_x = {}
        pcc_y= {}
        dis_error = {}
        for i in range(4):   
            reg_x = reg.fit(feature_all_train[i][:,7:18],x_train[i])
            regpre_x[i] = reg_x.predict(feature_all_test[i][:,7:18])
            reg_y = reg.fit(feature_all_train[i][:,7:18],y_train[i])
            regpre_y[i] = reg_y.predict(feature_all_test[i][:,7:18])

     
            r2_x[i] = r2_score(regpre_x[i],x_test[i])
            pcc_x[i] = pearsonr(regpre_x[i],x_test[i])
            r2_y[i] = r2_score(regpre_y[i],y_test[i])
            pcc_y[i] = pearsonr(regpre_y[i],y_test[i])
    

            dis_x = {}
            dis_y = {}
            dis = {}
            for j in range(50):
                dis_y[j] = regpre_y[i][j] - y_test[i][j]
                dis_x[j] = regpre_x[i][j] - x_test[i][j]
                dis[j] = (dis_x[j]**2 + dis_y[j]**2)**(0.5)
            sum = np.array([])
            for j in range(50):
                sum = np.concatenate((sum,dis[j]))
            dis_error[i] = np.mean(abs(sum))
        sum = 0 
        for i in range(4):
            sum = sum+dis_error[i]
        average_dis_error[k][m] = sum/4.0

In [10]:
kr_error_table = pd.DataFrame.from_dict({(i): average_dis_error[i]
                           for i in average_dis_error.keys() 
                          },
                       orient='index')

In [11]:
kr_error_table

,0,1,2,3,4,5,6,7,8,9,-1,-10,-9,-8,-7,-6,-5,-4,-3,-2
-10,163.912221,163.590748,163.270850,162.952520,162.635761,162.320577,162.006978,161.694973,161.384576,161.075796,164.235281,167.223071,166.882771,166.544886,166.209237,165.875668,165.544051,165.214279,164.886268,164.559951
-9,164.014419,163.693287,163.373733,163.055749,162.739333,162.424488,162.111221,161.799542,161.489462,161.180992,164.337147,167.323465,166.983092,166.645214,166.309642,165.976214,165.644790,165.315256,164.987517,164.661499
-8,164.115579,163.794777,163.475559,163.157914,162.841834,162.527323,162.214385,161.903028,161.593262,161.285098,164.437988,167.423093,167.082610,166.744699,166.409168,166.075846,165.744587,165.415264,165.087774,164.762037
-7,164.215727,163.895244,163.576353,163.259037,162.943289,162.629106,162.316492,162.005454,161.695999,161.388140,164.537831,167.521988,167.181358,166.843377,166.507850,166.174600,165.843473,165.514333,165.187071,164.861594
-6,164.314890,163.994712,163.676138,163.359144,163.043720,162.729861,162.417566,162.106842,161.797696,161.490137,164.636703,167.620181,167.279372,166.941284,166.605723,166.272510,165.941482,165.612497,165.285436,164.960197
-5,164.413092,164.093208,163.774939,163.458258,163.143151,162.829609,162.517629,162.207216,161.898374,161.591114,164.734631,167.717697,167.376681,167.038453,166.702823,166.369610,166.038647,165.709787,165.382899,165.057877
-4,164.510361,164.190755,163.872779,163.556402,163.241604,162.928372,162.616702,162.306595,161.998055,161.691089,164.831643,167.814560,167.473314,167.134916,166.799183,166.465935,166.135003,165.806234,165.479492,165.154661
-3,164.606721,164.287378,163.969683,163.653599,163.339101,163.026173,162.714807,162.405001,162.096759,161.790085,164.927766,167.910789,167.569294,167.230702,166.894835,166.561518,166.230583,165.901872,165.575246,165.250578
-2,164.702200,164.383101,164.065673,163.749871,163.435664,163.123032,162.811964,162.502455,162.194506,161.888121,165.023027,168.006402,167.664643,167.325835,166.989807,166.656390,166.325418,165.996734,165.670190,165.345659
-1,164.796822,164.477951,164.160773,163.845240,163.531314,163.218970,162.908193,162.598976,162.291316,161.985217,165.117455,168.101415,167.759378,167.420338,167.084127,166.750582,166.419542,166.090850,165.764358,165.439933


In [12]:
kr_error_table.to_csv('kr_error_table1.csv')